In [1]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
#from gurobipy import *

### Read in Excel data
Data preprocessing:
* Reset row names to "Site"
* Drop columns where mutation is on ChrX or ChrY 
* Remove starred sample names (but retain them in a seperate array named `stars`)

In [2]:
pnas = pandas.ExcelFile("pnas.1519556112.xlsx")
S3 = pnas.parse('s3_fill')
S3 = S3.set_index('Site')

for col in S3: 
    if "chrX" in col or "chrY" in col:
        S3 = S3.drop(col, axis=1)

stars = []
for site in S3.index:
    if '*' in site:
        stars.append(site)
        S3.rename({site: site.replace("*", "")}, inplace = True) 


# stars = np.array(['D9*', 'B17*', 'B19*', 'B20*', 'Z1*', 'A3*', 'C78*' ,'D6*'])


### Get coordinates from tumor map 
* Manually overlayed a `43 x 41` grid on the tumor map 
* Map coordinates from grid to dictionary 

In [3]:
coordinates = {
    "A68": [(15,0)],
     "A2": [(19,0), (20,0)],
    "A58": [(17,1)],
    "A67": [(13,1), (13,2)],
    "A57": [(15, 2), (16, 2)],
    "A47": [(18, 3), (19, 3)],
    "A48": [(20, 2), (20, 3)],
    "A70": [(4,3), (5,3), (4,4), (5,4)],
    "A69": [(7,4)],
    "A66": [(10, 3), (10, 4), (11, 3), (11, 4)],
    "A56": [(13, 4)],
    "A65": [(8, 5), (9, 5)],
    "A55": [(11, 5), (12, 5)],
    "A46": [(17, 4), (17, 5)],
    "A38": [(19, 5), (19, 6), (20, 5), (20, 6)],
    "A45": [(14, 6), (15, 6)],
    "A37": [(17, 6), (17, 7), (18, 6), (18, 7)],
    "A29": [(19, 7), (19, 8), (20, 7), (20, 8)],
    "A64": [(7, 7)],
    "A54": [(10, 7)],
    "A36": [(16, 8)],
    "A28": [(18, 9)],
    "A63": [(6,9)],
    "A53": [(9,9), (9, 10)],
    "A44": [(11, 10)],
    "A35": [(14, 10), (15, 10)],
    "A27": [(16, 10), (17, 10)],
    "A22": [(19, 10), (20, 10)],
    
    "A62": [(5, 11)],
    "A52": [(7,11), (7,12)],
    "A43": [(9, 11), (10, 11)],
    "A34": [(13,11)],
    "A21": [(18,11)],
    "A26": [(15,12)],
    "A20": [(16,12),(16,13),(17,12),(17,13)],
    "A61": [(4,13),(5,13)],
    "A42": [(8,13)],
    "A33": [(11,12), (11,13)],
    "A25": [(13,13),(14,13)],
    "A51": [(6,14)],
    "A32": [(10,14)],
    "A60": [(3,15),(4,15)],
    "A41": [(8,15)],
    "A24": [(12,15),(13,15)],
    "A19": [(15,14),(15,15)],
    "A13": [ (17,15)],
    "A14": [(18,13),(18,14)],
    "A15": [(19,12),(19,13)],
     "A9": [(19,14), (19,15)],
    "A50": [ (5,16),(5,17)],
    "A31": [(9,16)],
    "A18": [(13,16),(14,16)],
    "A12": [(16,16)],
     "A8": [(18,16)],
     "A5": [(20,15),(20,16)],
    "A59": [(3,17),(4,17)],
    "A40": [(6,17),(6,18),(7,17),(7,18)],
    "A23": [(11,17)],
    "A17": [(13,27), (13, 18)],
    "A11": [(15,17)],
     "A7": [(17,17)],
     "A4": [(19,17)],
     "A1": [(3,19), (3, 20)],
    "A49": [(4, 18), (4, 19)],
    "A39": [(6, 19)],
    "A30": [(8,18), (8,19)],
    
    "D55": [(3,22), (3,23)],
    "D43": [(4,23)],
    "D33": [(6,22), (6,23)],
    "D24": [(8,22),(8,23),(9,22),(9,23)],
    "D18": [(11,22),(11,23)],
    "D13": [(13,22)],
     "D9": [(16,21)],
     "D8": [(17,22), (17,23)],
    "D54": [(3,25),(4, 25)],
    "D42": [(5,25),(6,25)],
    "D32": [(7,24),(8,24)],
    "D12": [(15,24),(16,24)],
     "D7": [(19,24)],
    "D53": [(5,26)],
    "D41": [(7,26)],
    "D31": [(8,25),(8,26),(9,25),(9,26)],
    "D23": [(11,25)],
    "D17": [(13,25),(14,25)],
    "D53": [(5,26)],
     "D6_": [(3,27), (3, 28),(4,27),(4,28)],
    "D30": [(10,27)],
    "D22": [(12,26)],
    "D16": [(15,26),(15,27),(16,26),(16,27)],
    "D10": [(19,27),(20,27)],
    "D11": [(17,25),(17,26),(18,25),(18,26)],
    "D52": [(5,28),(6,28),(6,29)],
    "D40": [(8,28),(9,28)],
    "D29": [(12,28)],
     "D2": [(13,27),(13,28)],
     "D1": [(15,28)],
    "D15": [(18,28)],
     "D5": [(4,29)],
    "D51": [(7,29),(7,30)],
    "D39": [(10,29),(10,30),(11,29),(11,30)],
    "D28": [(13,29),(13,30),(14,29),(14,30)],
    "D21": [(16,29)],
    "D14": [(20,29)],
    
    # updated D6 manually because it was a repeat of D62
    "D6": [(4, 30), (5, 30)], 
    "D50": [(9,31)],
    "D38": [(12,31)],
    "D27": [(15,30),(15,31),(16,30),(16,31)],
    "D20": [(18,30)],
    "D19": [(19,31),(20,30),(20,31)],
    "D62": [(4,32),(5,32)],
    "D67": [(7,31),(7,32)],
    "D49": [(10,32),(10,33),(11,32),(11,33)],
    "D37": [(14,32)],
    "D26": [(17,32)],
    "D61": [(6,33),(6,34)],
    "D66": [(8,33),(9,33)],
    "D46": [],
    "D36": [(15,33),(15,34),(15,33),(16,34)],
    "D25": [(20,33),(21,33)],
    "D60": [(8,35)],
    "D65": [(10,34)],
    "D48": [(12,34),(13,34)],
    "D47": [(14,35)],
    "D35": [(17,34),(18,34)],
    "D59": [(10,36)],
    "D46": [(16,36)],
     "D4": [(11,36), (11, 37), (12, 36), (12, 37)],
    "D64": [(12, 35)],
    "D45": [(18,36),(18,37)],
    "D44": [(20,37)],
    "D34": [(20,35)],
     "D3": [(13,37),(13,38),(14,37),(14,28)],
    "D58": [(15,38),(15,39)],
    "D57": [(17,38),(17,39), (18,38),(18,39)],
    "D56": [(19,39),(19,40),(20,39),(20,40)],   
    
    
     "B3": [(27,1)],
    "B83": [(25,1),(25,2)],
     "B4": [(29,2)],
    "B55": [(24,3)],
    "B56": [(26,3),(26,4)],
    "B64": [(27,3),(28,3)],
    "B65": [(29,3),(29,4),(30,3),(30,4)],
    "B48": [(23,5),(24,5)],
    "B57": [(27,5),(28,5)],
    "B58": [(30,6)],
    "B47": [(25,5),(25,6),(26,5),(26,6)],
    "B66": [(31,4),(31,5)],
    "B67": [(32,6),(33,6)],
    "B37": [(23,7)],
    "B48_": [(27,7)],
    "B69": [(35,6),(35,7)],
    "B68": [(24,7)],
    "B38": [(25,8),(26,8)],
    "B49": [(29,8)],
    "B59": [(32,7),(31,8),(32,7)],
    "B32": [(23,9)],
    "B39": [(27,9)],
    "B50": [(31,9)],
    "B60": [(33,9),(34,9)],
     "B5": [(35,8),(36,8),(35,9),(36,9)],
    "B33": [(25,9),(25,10)],
     "B6": [(36,10),(37,10)],
    "B40": [(29,10)],
    
    
    "B26": [(23,10),(23,11)],
    "A16": [(21,11)],
     "B7": [(26,11),(27,11)],
    "B51": [(32,10),(32,11)],
    "B61": [(35,10),(35,11)],
    "B27": [(25,11),(25,12)],
     "B8": [(28,11),(28,12)],
    "B41": [(30,11),(30,12),(31,11),(31,12)],
    "B62": [(37,11),(37,12)],
    "B20": [(23,12),(23,13),(24,12),(24,13)],
    "B28": [(26,12),(26,13)],
    "B52": [(34,12),(35,12)],
     "B9": [(29,13),(30,13)],
    "A10": [(21,13),(21,14)],
     "B1": [(27,13),(28,13)],
    "B21": [(25,13),(25,14)],
    "B10": [(30,14)],
     "B2": [(28,14),(29,14)],
    "B42": [(32,13),(33,13)],
    "B53": [(36,13),(36,14)],
    "B15": [(23,14)],
    "B22": [(26,14),(26,15),(27,14),(27,15)],
    "B34": [(14,31),(15,31)],
    "B43": [(34,14)],
    "B54": [(38,14)],
     "A6": [(21,15),(22,15)],
    "B16": [(24,15),(25,15)],
    "B17": [(26,16)],
    "B23": [(28,15),(27,16),(28,16)],
    "B29": [(29,15),(30,15)],
    "B35": [(33,16)],
    "B44": [(36,16)],
     "A3": [(21,16),(21,17)],
    "B12": [(23,16),(23,17)],
    "B13": [(24,17),(25,17)],
    "B18": [(27,17)],
    "B24": [(29,16),(29,17)],
    "B30": [(30,16),(31,17),(31,16)],
    "B36": [(34,17),(35,17)],
    "B45": [(37,17),(38,17),(37,18)],
    "B31": [(32,17),(32,18)],
    "B11": [(24,18)],
    "B14": [(26,18)],
    "B19": [(28,18)],
    "B25": [(30,18)],
     "Z1": [(21,19),(22,19),(21,20),(22,20)],
    
    "C82": [(24,21)],
    "C85": [(26,21)],
    "C89": [(29,21),(30,21)],
     "C4": [(32,21)],
     "C9": [(34,21),(35,21)],
    "C17": [(36,21),(36,22)],
     "C1": [(41,21),(41,22)],
    "C86": [(26,22)],
     "C5": [(31,22),(31,23)],
    "C10": [(33,22),(33,23),(34,22),(34,23)],
    "C28": [(38,22),(38,23)],
    "C83": [(24,23)],
    "C90": [(28,23),(28,24), (29,23), (29,24)],
    "C18": [(35,23),(35,24),(36,23)],
    "C27": [(37,24)],
    "C37": [(39,23),(40,23)],
    "C48": [(42,23)],
    "C11": [(33,24)],
    "C84": [(23,25),(24,25)],
    "C87": [(25,25),(26,25)],
    "C19": [(34,25)],
    "C38": [(38,25),(39,25)],
    "C49": [(40,25),(41,25)],
    "C80": [(42,25), (43,25)],
    "C28": [(36,25),(36,26)],
     "C6": [(29,25),(29,26)],
    "C88": [(24,27)],
     "C3": [(27,26),(27,27)],
    "C12": [(30,27),(31,26),(31,27)],
    "C29": [(35,27)],
    "C39": [(37,26), (37,27)],
    "C50": [(39,26), (39,27), (40,26), (40,27)],
    "C51": [(38,28),(38,29),(39,28),(39,29)],
     "C7": [(26,28)],
    "C13": [(29,28)],
    "C20": [(31,28),(32,28)],
    "C30": [(34,28), (34,29)],
    "C40": [(36,28),(36,29)],
    "C62": [(40,28),(40,29)],
    "C72": [(42,28),(42,29)],
     "C8": [(24,29),(24,30)],
    "C14": [(27,29)],
    "C21": [(30,29)],
    "C31": [(32,29), (32,30),(33,29), (33,30)],
    "C41": [(35,30)],
    "C61": [(41,27)],
    
    "C52": [(37,30)],
    "C15": [(26,30),(26,31)],
    "C22": [(28,30),(28,31),(29,30),(29,31)],
    "C63": [(38,30),(38,31),(39,30),(39,31)],
    "C73": [(40,30),(40,31),(41,30),(41,31)],
    "C32": [(31,31)],
    "C42": [(33,31),(33,32)],
    "C23": [(27,31),(27,32)],
    "C53": [(35,31),(35,32)],
    "C16": [(24,32)],
    "C24": [(26,32),(26,33)],
    "C33": [(29,32)],
    "C64": [(37,32)],
    "C74": [(39,32),(40,32)],
    "C43": [(31,33)],
    "C34": [(28,33)],
    "C54": [(33,33),(34,33)],
    "C65": [(36,33),(36,34)],
    "C25": [(24,34),(25,34)],
    "C44": [(29,34),(30,34)],
    "C35": [(26,34)],
    "C75": [(37,34),(38,34)],
    "C45": [(28,35)],
    "C55": [(31,34),(31,35),(32, 34),(32,35)],
    "C68": [(35,35)],
    "C36": [(24,35), (24,36), (25,35), (25,36)],
    "C56": [(30,35),(30,36)],
    "C67": [(33,36)],
    "C76": [(36,35),(36,36)],
    "C46": [(26,36),(27,36)],
    "C47": [(24,37),(25,37)],
    "C57": [(28,36),(28,37)],
    "C68": [(31,37)],
    "C77": [(34,36),(34,37),(35,36),(35,37)],
    "C58": [(26,38),(27,38)],
    "C69": [(29,38)],
    "C78": [(33,37),(33,38)],
    "C59": [(25,38),(25,39)],
    "C79": [(31,38),(31,39)],
    "C70": [(27,39)],
    "C80": [(29,39)],
    "C71": [(25,40),(26,40)],
    "C81": [(27,39),(27,40)],
     "C2": [(25,41)]
}

### Read in samples from excel sheet 

* `sites` = sites where biopsies were performed; take away the two header values 
* `samples` = sites that are present in both the excel and mapped value 

In [4]:
sites = list(S3.index)[2:]

In [5]:
# samples are present in both the tumor map and the excel sheet s3
samples = []

print("These values are not found in excel sheet: ")
for key in coordinates:
    if key in sites and key not in samples:
        samples.append(key)
    else:
        # found on map but NOT in excel sheet!! 
        print(key)


# map row index to corresponding sample
samplesIdx = {}
for i in range(len(samples)):
    samplesIdx[i] = samples[i]

print("\n\nThese values are not found in the map:") 
for key in sites: 
    if key not in samples:
        print(key)

These values are not found in excel sheet: 
A55
A54
A18
D6_
B83
B48_
B28
B1
C89
C83
C90
C88
C22
C32


These values are not found in the map:
B46
B63
C66
Z2
D63
C26
Normal
C60(Normal)


In [6]:
len(samples)

280

### Read in mutation frequencies 


In [7]:
mutations = S3.columns

In [8]:
# map mutation to column in matrix F
mutCol = {}
idx = 0
for mut in mutations:
    mutCol[mut] = idx
    idx+=1

# Give each mutation locus an index 
revMutCol = {}
idx = 0
for key in mutCol:
    revMutCol[idx] = key
    idx+=1 

### Fill in grid 

* Helper functions used to determine corners of grid, making those values 0

In [9]:
def isInCircle(x,y): 
    # center is (22, 20)
    radius = 22
    dist = np.sqrt((y - (20+1))**2 + (x - (22+1) )**2 )
    return dist < radius

In [10]:
# 43 columns 
# 41 rows 

nRows = 41 +1
nCols = 43 +1

f = np.full((nRows*nCols,len(mutations)), np.nan)
blank = ['-', 'P', 'F']

### Fill in Matrix F 

This method does NOT interpolate. Here, we only fill the empty grid with values from the PNAS spreadsheet, where they exist. 
* Values outside of clone regions that are in `-,F,P` will be marked as 0 
* Values outside of the circular region of the grid will be marked as 0
* All other values are marked as NaN for interpolation later

In [11]:
for mut in mutations: 
    print("Calculating mutation " + mut)
    grid = np.full((nRows,nCols), np.nan)
    # fill square - circle with 0's for interpolation
    for row in range(0, len(grid)):
        for col in range(0,len(grid[0])):
            if(not isInCircle(col,row)):
                grid[row][col] = 0.0

    grid[0:5,2:10] = np.nan
    
    # create grid from coordinates 
    for site in samples:
        coords = coordinates[site]
        for pair in coords:
            x = pair[0]
            y = pair[1]
            if(S3[mut][site] in blank):
                grid[y][x] = 0
            else:
                grid[y][x] = float(S3[mut][site])
                
    # Pad the borders with 0 for interpolation
    grid = np.insert(grid, 0, values=np.zeros(nCols), axis=0)
    grid = np.insert(grid, len(grid), values = np.zeros(nCols), axis=0)
    grid = np.insert(grid, 0, values = np.zeros(nRows+2), axis=1)
    grid = np.insert(grid, len(grid[0]), values = np.zeros(nRows+2), axis=1)
    df = pandas.DataFrame(grid)
    
#     row = df.interpolate(method='linear', limit_direction='both', axis=1)
#     col = df.interpolate(method='linear', limit_direction='both', axis=0)
#     avg = (row + col) / 2
    
    interpolated = df

    # ROW-MAJOR 
    # add excluding the 0 borders 
    for row in range(1, len(interpolated)-1):
        for col in range(1, len(interpolated[0])-1):
            rowIdx = (col-1) + ((row-1)*(nCols))
#             print(row, col, rowIdx, mutCol[mut], mut,interpolated[row][col])
            f[rowIdx][mutCol[mut]] = interpolated[row][col]

Calculating mutation chr16_21698785
Calculating mutation chr16_69988362
Calculating mutation chr2_170031901
Calculating mutation chr3_58155491
Calculating mutation chr10_101969461
Calculating mutation chr11_118375433
Calculating mutation chr8_8998522
Calculating mutation chr21_46306293
Calculating mutation chr8_104897577
Calculating mutation chr17_42882927
Calculating mutation chr19_13563795
Calculating mutation chr3_47951371
Calculating mutation chr15_90145059
Calculating mutation chr9_5090748
Calculating mutation chr18_11851559
Calculating mutation chr17_42824879
Calculating mutation chr7_45122631
Calculating mutation chr7_158829499
Calculating mutation chr12_6427505
Calculating mutation chr19_9075018
Calculating mutation chr2_76976008
Calculating mutation chr7_99227274
Calculating mutation chr3_32746404
Calculating mutation chr7_645878
Calculating mutation chr17_38240142
Calculating mutation chr9_123629149
Calculating mutation chr7_72913004
Calculating mutation chr6_108370471
Calcul

In [12]:
df=pandas.DataFrame(f)

## Name samples

In [13]:
# for row in range(len(f)):
#     for col in range(len(f[0])):
#         if f[row][col] == 'nan':
#             print(str(row) + " " + str(col))

# rename columns 
for col in df.columns:
    df = df.rename(columns={col: revMutCol[col]})

#rename rows
rowNames = np.full((nRows*nCols), "Interpolated")

for site in samples:
    coords = coordinates[site]
    for pair in coords:
        x = pair[0]
        y = pair[1]
        rowIdx = ((x)*nCols + (y))
        try:
            rowNames[rowIdx] = site
        except:
            print(x,y,rowIdx,site)

rowNames.shape
np.set_printoptions(threshold=np.nan)
rowNames

for row in df.index:
    if rowNames[row] != "None":
        df = df.rename(index={row: rowNames[row]})

42 23 1871 C48
42 28 1876 C72
42 29 1877 C72


In [14]:
# Sanity check: does the DF actually contain values we would expect? 

# MLL (chr11_118375433) @ A2 should be: 0.528986
df['chr11_118375433']['A2']

A2    0.528986
A2    0.528986
Name: chr11_118375433, dtype: float64

## SNVs to exclude:

* Criterion: at least one sample with VAF > 0.6

All other SNVs with VAFs > 0.5 will be clipped to 0.5. Then, we multiply values by 2 to get cancer cell fractions

In [15]:
retained_samples = set(df.columns)
removed_samples = []
for col in df.columns:
    max_val = df[col].max()
    if max_val > 0.6:
        print(col, max_val, len(df[df[col] > 0.6]))
        retained_samples.remove(col)
        removed_samples.append(col)

df = df[retained_samples]

# make sure maximum is 0.5 (clip)
df = df.clip(lower=None, upper=0.5)

# multiply by 2 to get cancer cell fractions
df = df * 2

df.to_csv('matrixF-processed-locus.csv', index=True, sep=',', header=True)

chr16_21698785 0.793417 35
chr16_69988362 0.84691 4
chr17_42882927 0.617388 2
chr18_11851559 0.608778 2
chr12_6427505 0.865647 2
chr6_108370471 0.681733 10


In [16]:
#removed_samples = ['chr16_21698785', 'chr16_69988362', 'chr17_42882927', 'chr18_11851559', 'chr12_6427505' , 'chr6_108370471']
print("Removed samples:")
for locus in removed_samples:
    print(S3[locus]['Gene'])
sex_chromosomes = ['HEPH', 'ZMAT1', 'MSN']
print()
for gene in sex_chromosomes:
    print(gene)

Removed samples:
OTOA
CLEC18A
GJC1
CHMP1B
PLEKHG6
OSTM1

HEPH
ZMAT1
MSN


In [17]:
len(retained_samples)
locusToGene = {}
for locus in retained_samples: 
    locusToGene[locus] = S3[locus]['Gene']
# rename columns 
for col in df.columns:
    df = df.rename(columns={col: locusToGene[col]})
#df.to_csv('matrixF-processed-gene.csv', index=True, sep=',', header=True)

### Adding fixed mutations 
* The fixed mutations will be set to the max of each row 
* There are 206 unique fixed (trunk) mutations 
* 3 trunk mutations were repeated in the original PNAS data (under fixed mutations)

In [18]:
S2 = pnas.parse('s2')

In [19]:
# Check for overlaps between trunk mutations 
fixedMutations = []
print("Repeated trunk mutations: ")
for trunk in S2['Gene']:
    if trunk not in fixedMutations:
        fixedMutations.append(trunk)
    else: 
        print(trunk)
print("Number of trunk mutations: ", str(len(fixedMutations)))

for trunk in fixedMutations:
    for mut in retained_samples:
        if trunk == locusToGene[mut]:
            print(trunk)


Repeated trunk mutations: 
ODZ3
CDH26
HYDIN
Number of trunk mutations:  206


In [20]:
# get max of each row 
row_max = df.max(axis=1)

# assign max of each row to all trunk mutations;
# append to df
numTrunks = 0
for trunk in S2['Gene']:
#     print(trunk)
#     temp = pandas.DataFrame({'CPXM2': row_max})
    df[trunk] = row_max
    numTrunks+=1 
df.shape

(1848, 232)

### Getting the phylogeny tree 

* Matrix F has been computed, and processed above. 

In [21]:
#     ('Normal'): ['CPXM2', 'UBR4', 'SLC30A2', 'TRIM63', 'OR4C13', 'NRXN2', 'CLPB', 'DIXDC1', 'TMPRSS13', 'C2CD2L', 'OR8G1', 'FLG', 'AQP10', 'FAM189B', 'CD48', 'TNR', 'LAMC1', 'OBSCN', 'PABPC3', 'OR4K17', 'FUT8', 'RYR3', 'FAH', 'TTC23', 'KIAA0100', 'STXBP4', 'COG1', 'ZNF846', 'ZNF441', 'CD93', 'PPP1R16B', 'ADCY3', 'TTN', 'DNAH7', 'SPHKAP', 'HTR2B', 'PLA2G3', 'TGFBR2', 'DBR1', 'TLR6', 'DCHS2', 'ODZ3', 'ABCC10', 'TFAP2D', 'GJA10', 'AMD1', 'ULBP2', 'SAMD9', 'CCDC136', 'ASPH', 'GRHL2', 'TNFRSF11B', 'FREM1', 'PRUNE2', 'MAPKAP1', 'RALGDS', 'ARAF', 'FN3KRP', 'DOCK3', 'CD1C', 'ITGAX', 'COL11A2', 'SLC24A2', 'SCN4A', 'MYT1L', 'KIAA1731', 'SLC8A3', 'DLX6', 'CERCAM', 'KIAA2022', 'ZFYVE26', 'GPM6A', 'C3orf77', 'ELAC2', 'KIF5C', 'FCHSD1', 'UQCC', 'KIF15', 'LDLRAD1', 'UBE3B', 'TCEB3B', 'CXorf1', 'ADCY2', 'ISCA2', 'NEDD4', 'THSD1', 'ALDH7A1', 'TRPV2', 'HCRTR2', 'TSC1', 'IPO13', 'CASQ2', 'WDR35', 'F13A1', 'ITGB8', 'C7orf46', 'OSBPL6', 'WDR12', 'THOC7', 'PDE1C', 'ATP13A4', 'EIF4G3', 'MPP7', 'CAPN14', 'KLF5', 'GATAD2A', 'SNUPN', 'TRAF6', 'CDH26', 'ZEB2', 'CCDC80', 'USP29', 'GTPBP3', 'FAM181A', 'CD2AP', 'EDDM3B', 'GPR112', 'BEST3', 'FYTTD1', 'MUC4', 'LTBP2', 'CRX', 'PFKFB4', 'PHLDB2', 'AMMECR1L', 'SETD2', 'PIK3CG', 'MLL2', 'MAP3K5', 'PMM1', 'SDK1', 'CGNL1', 'HCN1', 'C3orf58', 'ATP10A', 'PREX1', 'KIAA1841', 'ANO4', 'PDE4DIP', 'SS18L1', 'LRFN5', 'HSPG2', 'ACSS1', 'E2F7', 'SLC6A3', 'AGAP4', 'TSPAN16', 'SMU1', 'ARHGAP21', 'AIM2', 'DNAH10', 'FAM55B', 'GRIN2A', 'ACACA', 'MYO7B', 'GPR144', 'FBXO41', 'DGKD', 'AKTIP', 'THEG', 'SPRYD3', 'C17orf42', 'SNX14', 'TP53', 'PLEKHG2', 'TRAPPC9', 'SYNE1', 'MAMDC4', 'ZAN', 'POMZP3', 'HYDIN', 'NTS', 'CASC1', 'KIAA1804', 'C15orf32', 'CENPF', 'GCNT7', 'RNF213', 'TBC1D5', 'MGAM', 'LRBA', 'CCAR1', 'MYBPC3', 'UBXN4', 'USP33', 'PLEKHA6', 'DEF6', 'ARHGEF12', 'ZFP14', 'AKIRIN1', 'ROD1', 'MAP2', 'DNAH8', 'TTLL3', 'LMX1B', 'TLE1', 'STRA6', 'SSC5D', 'IGF1R', 'UCHL5', 'U2AF1', 'FRMPD1', 'CASC3', 'DSCAM', 'ENPP3', 'COL21A1'],
tree = {
    ('CPXM2', 'UBR4', 'SLC30A2', 'TRIM63', 'OR4C13', 'NRXN2', 'CLPB', 'DIXDC1', 'TMPRSS13', 'C2CD2L', 'OR8G1', 'FLG', 'AQP10', 'FAM189B', 'CD48', 'TNR', 'LAMC1', 'OBSCN', 'PABPC3', 'OR4K17', 'FUT8', 'RYR3', 'FAH', 'TTC23', 'KIAA0100', 'STXBP4', 'COG1', 'ZNF846', 'ZNF441', 'CD93', 'PPP1R16B', 'ADCY3', 'TTN', 'DNAH7', 'SPHKAP', 'HTR2B', 'PLA2G3', 'TGFBR2', 'DBR1', 'TLR6', 'DCHS2', 'ODZ3', 'ABCC10', 'TFAP2D', 'GJA10', 'AMD1', 'ULBP2', 'SAMD9', 'CCDC136', 'ASPH', 'GRHL2', 'TNFRSF11B', 'FREM1', 'PRUNE2', 'MAPKAP1', 'RALGDS', 'ARAF', 'FN3KRP', 'DOCK3', 'CD1C', 'ITGAX', 'COL11A2', 'SLC24A2', 'SCN4A', 'MYT1L', 'KIAA1731', 'SLC8A3', 'DLX6', 'CERCAM', 'KIAA2022', 'ZFYVE26', 'GPM6A', 'C3orf77', 'ELAC2', 'KIF5C', 'FCHSD1', 'UQCC', 'KIF15', 'LDLRAD1', 'UBE3B', 'TCEB3B', 'CXorf1', 'ADCY2', 'ISCA2', 'NEDD4', 'THSD1', 'ALDH7A1', 'TRPV2', 'HCRTR2', 'TSC1', 'IPO13', 'CASQ2', 'WDR35', 'F13A1', 'ITGB8', 'C7orf46', 'OSBPL6', 'WDR12', 'THOC7', 'PDE1C', 'ATP13A4', 'EIF4G3', 'MPP7', 'CAPN14', 'KLF5', 'GATAD2A', 'SNUPN', 'TRAF6', 'CDH26', 'ZEB2', 'CCDC80', 'USP29', 'GTPBP3', 'FAM181A', 'CD2AP', 'EDDM3B', 'GPR112', 'BEST3', 'FYTTD1', 'MUC4', 'LTBP2', 'CRX', 'PFKFB4', 'PHLDB2', 'AMMECR1L', 'SETD2', 'PIK3CG', 'MLL2', 'MAP3K5', 'PMM1', 'SDK1', 'CGNL1', 'HCN1', 'C3orf58', 'ATP10A', 'PREX1', 'KIAA1841', 'ANO4', 'PDE4DIP', 'SS18L1', 'LRFN5', 'HSPG2', 'ACSS1', 'E2F7', 'SLC6A3', 'AGAP4', 'TSPAN16', 'SMU1', 'ARHGAP21', 'AIM2', 'DNAH10', 'FAM55B', 'GRIN2A', 'ACACA', 'MYO7B', 'GPR144', 'FBXO41', 'DGKD', 'AKTIP', 'THEG', 'SPRYD3', 'C17orf42', 'SNX14', 'TP53', 'PLEKHG2', 'TRAPPC9', 'SYNE1', 'MAMDC4', 'ZAN', 'POMZP3', 'HYDIN', 'NTS', 'CASC1', 'KIAA1804', 'C15orf32', 'CENPF', 'GCNT7', 'RNF213', 'TBC1D5', 'MGAM', 'LRBA', 'CCAR1', 'MYBPC3', 'UBXN4', 'USP33', 'PLEKHA6', 'DEF6', 'ARHGEF12', 'ZFP14', 'AKIRIN1', 'ROD1', 'MAP2', 'DNAH8', 'TTLL3', 'LMX1B', 'TLE1', 'STRA6', 'SSC5D', 'IGF1R', 'UCHL5', 'U2AF1', 'FRMPD1', 'CASC3', 'DSCAM', 'ENPP3', 'COL21A1'): [['MLL','CHUK'], ['GCK'],['KIAA0427','GABRA1'],['A2M'],['RIMS2'],['C15orf42','JAK2'],['MUC16'],['FLNB'],['LRP2']],
    ('MLL','CHUK'): [['PPP1R3B'], ['ITGB2']],
    ('RIMS2'): [['CACNA1A','MAP4']],
    ('C15orf42','JAK2'): [['NACAD','VIPR2'],['DBF4B']],
    ('MUC16'): [['ZNF498', 'LRRTM4']],
    ('FLNB'): [],
    ('LRP2'): [],
    ('ZNF498', 'LRRTM4'): [['BAZ1B'], ['CNOT10','PRKAR1B'], ['THRA']],
    ('PPP1R3B'): [],
    ('ITGB2'): [],
    ('GCK'): [],
    ('KIAA0427','GABRA1'): [],
    ('A2M'): [],
    ('CACNA1A','MAP4'): [],
    ('NACAD','VIPR2'): [],
    ('DBF4B'): [],
    ('BAZ1B'): [],
    ('CNOT10','PRKAR1B'): [],
    ('THRA'): [['PHF19']],
    ('PHF19'): [],
}

In [22]:
geneIndex = {}
# TODO: fix later? Root is currently starting at 1 in visualization 
idx = 0
for col in df: 
    geneIndex[col] = idx
    idx += 1

## Average mutation clusters
* Up until now, we have been working with mutations individually. However, some of them _always_ occur together in our tumor sample. Therefore, they will be combined by taking the average across all mutations in the cluster. 
* We will do this by traversing the `tree`, and averaging values in the nodes if it is > 1.  

In [23]:
averageDf = pandas.DataFrame()
averageDfLabels = pandas.DataFrame()
# # map pairs to a single column index 
# fIdx = {}

# map column index to gene pairs
idxToLabel = {}
colIdx = 1
for node in tree: 
    keys = []
    label = ""
    tempDf = pandas.DataFrame()
    
    # there is a pair 
    if type(node) == tuple:
        tempDf = 0
        for i in range(0,len(node)): 
            mut = node[i]
            label += mut + ","
            keys.append(mut)
            tempDf += df[mut]
        tempDf /= len(node)
        label = label[:-1]
        idxToLabel[colIdx] = keys
        
    else:
        tempDf = df[node]
        label = node
        idxToLabel[colIdx] = [node]
    averageDfLabels[label] = tempDf
    averageDf[colIdx] = tempDf
    colIdx += 1

In [24]:
labelToIdx = {}
for node in idxToLabel:
    labelToIdx[str(idxToLabel[node])] = node

In [25]:
newTree = {}
for node in tree: 
    newKey = str(node)
    if type(node) == tuple:
        newKey = "[" + newKey[1:-1] + "]"
    else:
        newKey = "['" + newKey + "']"
    
    newKey = labelToIdx[newKey]
    newVals = []
    for group in tree[node]: 

        newGroup = str(group)
        newVals.append(labelToIdx[newGroup])
    newTree[newKey] = newVals

In [26]:
jsonTree = {}
for node in tree: 
    key = ""
    if type(node) == tuple:
        for mut in node: 
            key += mut + ","
        key = key[:-1]
    else:
        key = node
    jsonTree[key] = tree[node]

In [27]:
# this is the numeric phylogeny that we will use in the JSON file for visualization. 
# this tree agrees with the original PNAS suggested tree. 

phylogeny = {"1": [2, 11, 12, 13, 3, 4, 5, 6, 7],
             "2": [9, 10],
             "3": [14],
             "4": [15, 16],
             "5": [8],
             "6": [],
             "7": [],
             "8": [17, 18, 19],
             "9": [],
             "10": [],
             "11": [],
             "12": [],
             "13": [],
             "14": [],
             "15": [],
             "16": [],
             "17": [],
             "18": [],
             "19": [20],
             "20": []
      }

In [28]:
idxTree = {}
for key in phylogeny:
    newVals = []
    for vals in phylogeny[key]:
        newVals.append(int(vals)-1)
    idxTree[int(key)-1] = newVals

In [29]:
averageDfLabels.head()

,"CPXM2,UBR4,SLC30A2,TRIM63,OR4C13,NRXN2,CLPB,DIXDC1,TMPRSS13,C2CD2L,OR8G1,FLG,AQP10,FAM189B,CD48,TNR,LAMC1,OBSCN,PABPC3,OR4K17,FUT8,RYR3,FAH,TTC23,KIAA0100,STXBP4,COG1,ZNF846,ZNF441,CD93,PPP1R16B,ADCY3,TTN,DNAH7,SPHKAP,HTR2B,PLA2G3,TGFBR2,DBR1,TLR6,DCHS2,ODZ3,ABCC10,TFAP2D,GJA10,AMD1,ULBP2,SAMD9,CCDC136,ASPH,GRHL2,TNFRSF11B,FREM1,PRUNE2,MAPKAP1,RALGDS,ARAF,FN3KRP,DOCK3,CD1C,ITGAX,COL11A2,SLC24A2,SCN4A,MYT1L,KIAA1731,SLC8A3,DLX6,CERCAM,KIAA2022,ZFYVE26,GPM6A,C3orf77,ELAC2,KIF5C,FCHSD1,UQCC,KIF15,LDLRAD1,UBE3B,TCEB3B,CXorf1,ADCY2,ISCA2,NEDD4,THSD1,ALDH7A1,TRPV2,HCRTR2,TSC1,IPO13,CASQ2,WDR35,F13A1,ITGB8,C7orf46,OSBPL6,WDR12,THOC7,PDE1C,ATP13A4,EIF4G3,MPP7,CAPN14,KLF5,GATAD2A,SNUPN,TRAF6,CDH26,ZEB2,CCDC80,USP29,GTPBP3,FAM181A,CD2AP,EDDM3B,GPR112,BEST3,FYTTD1,MUC4,LTBP2,CRX,PFKFB4,PHLDB2,AMMECR1L,SETD2,PIK3CG,MLL2,MAP3K5,PMM1,SDK1,CGNL1,HCN1,C3orf58,ATP10A,PREX1,KIAA1841,ANO4,PDE4DIP,SS18L1,LRFN5,HSPG2,ACSS1,E2F7,SLC6A3,AGAP4,TSPAN16,SMU1,ARHGAP21,AIM2,DNAH10,FAM55B,GRIN2A,ACACA,MYO7B,GPR144,FBXO41,DGKD,AKTIP,THEG,SPRYD3,C17orf42,SNX14,TP53,PLEKHG2,TRAPPC9,SYNE1,MAMDC4,ZAN,POMZP3,HYDIN,NTS,CASC1,KIAA1804,C15orf32,CENPF,GCNT7,RNF213,TBC1D5,MGAM,LRBA,CCAR1,MYBPC3,UBXN4,USP33,PLEKHA6,DEF6,ARHGEF12,ZFP14,AKIRIN1,ROD1,MAP2,DNAH8,TTLL3,LMX1B,TLE1,STRA6,SSC5D,IGF1R,UCHL5,U2AF1,FRMPD1,CASC3,DSCAM,ENPP3,COL21A1","MLL,CHUK",RIMS2,"C15orf42,JAK2",MUC16,FLNB,LRP2,"ZNF498,LRRTM4",PPP1R3B,ITGB2,GCK,"KIAA0427,GABRA1",A2M,"CACNA1A,MAP4","NACAD,VIPR2",DBF4B,BAZ1B,"CNOT10,PRKAR1B",THRA,PHF19
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Sanity check after averaging mutation clusters:
# ZNF498,LRRTM4 at D16 should be: (0.393512*2+0.469388*2)/2 = 0.8629

averageDfLabels['ZNF498,LRRTM4']['D16']

D16    0.8629
D16    0.8629
D16    0.8629
D16    0.8629
Name: ZNF498,LRRTM4, dtype: float64

# Interpolate! 
* Now, we have NaN values where there is no data from biopsies. 
* These will interpolate between two nearest known real data points, according to the values locations on the grid. 

* NOTE: We have manually removed NaN values in 3 clones to reduce some interpolation artifacts. 
* `['FLNB','NACAD,VIPR2','CACNA1A,MAP4']`

In [31]:
len(averageDfLabels['MLL,CHUK'])

1848

In [32]:
s3prime = S3
s3prime.columns = s3prime.iloc[0]

modTree_labels = {
#     ('trunk'): [['MLL','CHUK'], ['GCK'],['KIAA0427','GABRA1'],['A2M'],['RIMS2'],['C15orf42','JAK2'],['MUC16'],['FLNB'],['LRP2']],
    ('MLL,CHUK'): [['PPP1R3B'], ['ITGB2']],
    ('RIMS2'): [['CACNA1A,MAP4']],
    ('C15orf42,JAK2'): [['NACAD,VIPR2'],['DBF4B']],
    ('MUC16'): [['ZNF498', 'LRRTM4']],
    ('FLNB'): [],
    ('LRP2'): [],
    ('ZNF498,LRRTM4'): [['BAZ1B'], ['CNOT10,PRKAR1B'], ['THRA']],
    ('PPP1R3B'): [],
    ('ITGB2'): [],
    ('GCK'): [],
    ('KIAA0427,GABRA1'): [],
    ('A2M'): [],
    ('CACNA1A,MAP4'): [],
    ('NACAD,VIPR2'): [],
    ('DBF4B'): [],
    ('BAZ1B'): [],
    ('CNOT10,PRKAR1B'): [],
    ('THRA'): [['PHF19']],
    ('PHF19'): [],
}

labelToIdx_labels={"['trunk']":1,"['MLL,CHUK']": 2,
 "['RIMS2']": 3,
 "['C15orf42,JAK2']": 4,
 "['MUC16']": 5,
 "['FLNB']": 6,
 "['LRP2']": 7,
 "['ZNF498,LRRTM4']": 8,
 "['PPP1R3B']": 9,
 "['ITGB2']": 10,
 "['GCK']": 11,
 "['KIAA0427,GABRA1']": 12,
 "['A2M']": 13,
 "['CACNA1A,MAP4']": 14,
 "['NACAD,VIPR2']": 15,
 "['DBF4B']": 16,
 "['BAZ1B']": 17,
 "['CNOT10,PRKAR1B']": 18,
 "['THRA']": 19,
 "['PHF19']": 20}

In [34]:
missing_coords=[]
f = np.full((nRows*nCols,len(tree)), np.nan)

# Sanity check: to make sure all row names end up with the same values!
rowNames=[]

biopsy={}
for node in modTree_labels:
    keys = []
    grid = np.full((nRows,nCols), np.nan)
    # fill square - circle with 0's for interpolation
    for row in range(0, len(grid)):
        for col in range(0,len(grid[0])):
            if(not isInCircle(col,row)):
                grid[row][col] = 0.0
                
    grid[0:5,2:10] = np.nan
    
    mut = node
    keys.append(mut)
    for site in samples:
        coords = coordinates[site]
        for pair in coords:
            x = pair[0]
            y = pair[1]
            val=""
            k=str(y)+"_"+str(x)
            try:
                val=averageDfLabels[mut][site]
                biopsy[k]=site
            except:
                # there are 4 sites that get lost when converting to a grid. 
                # they are the same across all mutations. this is only printed once to reduce clutter
                if mut=='BAZ1B':
                    print(mut, site)
            
            try:
                if val=='':
                    val = 0
                val=float(val)
            except:
                val=float(val.values[0])

            grid[y][x]=val

    ## MANUAL CODE to limit interpolation for certain clones (due to artifacts)
    if mut in ['FLNB','NACAD,VIPR2','CACNA1A,MAP4', 'PPP1R3B', 'A2M']:
        grid=np.nan_to_num(grid)

    # Pad the borders with 0 for interpolation
    grid = np.insert(grid, 0, values=np.zeros(nCols), axis=0)
    grid = np.insert(grid, len(grid), values = np.zeros(nCols), axis=0)
    grid = np.insert(grid, 0, values = np.zeros(nRows+2), axis=1)
    grid = np.insert(grid, len(grid[0]), values = np.zeros(nRows+2), axis=1)
    df = pandas.DataFrame(grid)
    
    row = df.interpolate(method='linear', limit_direction='both', axis=1)
    col = df.interpolate(method='linear', limit_direction='both', axis=0)
    avg = (row + col) / 2
    
    interpolated = np.array(avg)
    
    temp_rownames=np.full((nRows*nCols), "Interpolated")
    # ROW-MAJOR 
    # add excluding the 0 borders 
    for row in range(1, len(interpolated)-1):
        for col in range(1, len(interpolated[0])-1):
            rowIdx = (col-1) + ((row-1)*(nCols))
            k=str(row-1)+"_"+str(col-1)
            f[rowIdx][labelToIdx_labels[str(keys)]-1] = interpolated[row][col]
            try:
                temp_rownames[rowIdx]=biopsy[k]
            except:
                # interpolated; no need for label
                pass
    rowNames.append(temp_rownames)

/usr/local/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


BAZ1B C48
BAZ1B C72
BAZ1B C72
BAZ1B C70


In [35]:
ff=pandas.DataFrame(f)

## Rename matrix

Also set the root equal to the max of each row. 

In [36]:
for idx in range(1,len(rowNames)):
    if not (rowNames[idx]==rowNames[idx-1]).all():
        print(str(idx), str(idx-1), " Indices do not match! ")

In [37]:
for row in ff.index:
    if rowNames[0][row] != "None":
        ff = ff.rename(index={row: rowNames[0][row]})

In [38]:
# set root to max of each row
ff[0]=ff.max(axis=1)

In [39]:
ff.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# Sanity check! Index 1 refers to MLL,CHUK. 
# (0.394624*2 + 1)/2 = 0.894624

# Note: MLL value is 1 because the original value of 0.528986 
# exceeds 0.5; thus it was capped at 0.5 before being multiplied by 2 

ff[1]['A2']

A2    0.894624
A2    0.894624
Name: 1, dtype: float64

In [46]:
ff[8]['B5']

B5    0.194
B5    0.194
B5    0.194
B5    0.194
Name: 8, dtype: float64

In [44]:
ff=ff.fillna(0)
ff.to_csv('new_fff_ppp.csv')

### Follow up with linear program in `Lp.ipynb`
* Compute a prevalence matrix U that best explains frequencies F, and is constrained by the original given phylogeny tree. 
* This ensures that interpolated values do not violate the original tree. 

# Recreate PNAS figure

* `grid_map.csv` is an excel sheet that corresponds to the mapping of the PNAS figure 

In [427]:
grid_map=pd.read_csv('./grid_map.csv',header=None)
grid_map=grid_map.fillna(0)

In [428]:
nRows = 41 +1
nCols = 43 +1
mutations=np.arange(1,8)

In [429]:
f = np.full((nRows*nCols,len(mutations)+1), 0.0)

In [430]:
def isInCircle(x,y): 
    # center is (22, 20)
    radius = 22
    dist = np.sqrt((y - (20+1))**2 + (x - (22+1) )**2 )
    return dist < radius

In [431]:
f.shape

(1848, 8)

In [432]:
root=0
# taken from data_prep.ipynb; how we turn grid into matrix F
interpolated=grid_map.values
# for each clone...
for i in range(1, 8):
    for row in range(1, len(interpolated)-1):
        for col in range(1, len(interpolated[0])-1):
            rowIdx = (col-1) + ((row-1)*(nCols))
            
            if int(interpolated[row][col])==i and isInCircle(row, col):
                f[rowIdx][i] = 0.5
                f[rowIdx][root] = 1.0
            else:
                f[rowIdx][i] = 0.0
                
            if i==1 and int(interpolated[row][col])==i:
                f[rowIdx][i] = 0.5
                f[rowIdx][root] = 1.0

In [433]:
np.sum(ff[0])

1369.0

In [434]:
count=0
for i in f:
    for j in i:
        if j != 0:
            count+=1
print(count)

2696


In [435]:
ff=pandas.DataFrame(f)

In [436]:
ff.to_csv('pnas_grid.csv',index=None)

# Misc.

This section contains old code

* Insert coordinates into matrix F first 
* Getting x,y,z to work with scipy.interp2d and scipy.griddata 

In [ ]:
grid = np.full((nRows,nCols), np.nan)
# fill square - circle with 0's for interpolation
for row in range(0, len(grid)):
    for col in range(0,len(grid[0])):
        if(not isInCircle(col,row)):
            grid[row][col] = 0.0

grid[0:6,2:10] = np.nan



In [357]:
f = np.full((1763,35), None)
nRows = 43
nCols = 41

for mut in mutations:
    for site in samples:
        coords = coordinates[site]
        for pair in coords:
            x = pair[0]
            y = pair[1]
            rowIdx = x + (y*nCols)
#            print("X: " + str(x) + " Y: " + str(y) + " rowIdx: " + str(rowIdx))
            f[rowIdx][mutCol[mut]] = S3[mut][site]

In [358]:
nones = 0
real = 0
for row in range(len(f)):
    for col in range(len(f[row])):
        if f[row][col] == None:
            nones+=1
        else:
#            print (f[row][col])
            real +=1
        if f[row][col] == 0.517516:
            print("Row: " + str(row) + " Col: " + str(col))
print("Number of None values: " + str(nones))
print("Number of real values: " + str(real))

Row: 195 Col: 5
Row: 236 Col: 5
Number of None values: 44873
Number of real values: 16832


In [ ]:
blank = ['-', 'F', 'P']
# x coordinates of real-valued cells 
x = []
# y coordinates of real-valued cells 
y = []
# values of real-valued cells
z = []
for row in range(len(grid)):
    for col in range(len(grid[0])):
        freq = grid[row][col]
        # if we have a good value, add to x,y,z
        if freq not in blank and not np.isnan(freq) :
            x.append(row)
            y.append(col)
            z.append(grid[row][col])
            

In [383]:
df = pandas.DataFrame(
    [(0.0,  np.nan, np.nan, np.nan, 0.0),
    (np.nan, -1.0, np.nan, np.nan, np.nan),
    (3.0, 3.0, -1.0, 0.2, np.nan),
    (np.nan, 4.0, -4.0, 16.0, np.nan)],
    columns=list('abcde'))

row_interp = df.interpolate(method='linear', limit_direction='both', axis=1)
col_interp = df.interpolate(method='linear', limit_direction='both', axis=0)
(row_interp + col_interp) / 2

,a,b,c,d,e
0,0.00,-0.5,-0.5,0.1,0.0
1,0.25,-1.0,-1.0,-0.4,-0.5
2,3.00,3.0,-1.0,0.2,0.1
3,3.50,4.0,-4.0,16.0,8.0


In [ ]:
grid = [[0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [0,   0,   0,   0,   0, 0.2,   0,   0,   0,   0],
        [0,   0,   0,   0, 0.3, 0.5, 0.3,   0,   0,   0],
        [0,   0,   0, 0.6, 0.7, 0.4, 0.2, 0.4,   0,   0],
        [0,   0, 0.8, 0.5, 0.4, 0.6, 0.2, 0.3, 0.6,   0],
        [0,   0, 0.2,   0, 0.3, 0.5, 0.7, 0.5, 0.6,   0],
        [0,   0,   0, 0.6, 0.7, 0.4, 0.2, 0.4,   0,   0],
        [0,   0,   0,   0, 0.3, 0.5,   0,   0,   0,   0],
        [0,   0,   0,   0,   0, 0.2,   0,   0,   0,   0],
        [0,   0,   0,   0,   0,   0,   0,   0,   0,   0]]
grid = np.asarray(grid, dtype = float) 
grid.shape

In [210]:
grid = np.full((nRows,nCols), np.nan)
print(grid.shape)

blank = ['-', 'P', 'F']
for site in samples:
    coords = coordinates[site]
    for pair in coords:
        x = pair[0]
        y = pair[1]
        if(S3['chr11_118375433'][site] in blank):
            grid[y][x] = np.nan
        else:
            grid[y][x] = float(S3['chr11_118375433'][site])
        

# Pad the borders with 0 for interpolation
grid = np.insert(grid, 0, values=np.zeros(nCols), axis=0)
grid = np.insert(grid, len(grid), values = np.zeros(nCols), axis=0)
grid = np.insert(grid, 0, values = np.zeros(nRows+2), axis=1)
grid = np.insert(grid, len(grid[0]), values = np.zeros(nRows+2), axis=1)

df = pandas.DataFrame(grid)
pandas.set_option('display.max_columns', 1000)

#grad.apply(pandas.to_numeric, errors='coerce')
#df.to_csv('chr11_118375433.csv', index=True, sep=',', header=True)

row = df.interpolate(method='linear', limit_direction='both', axis=1)
col = df.interpolate(method='linear', limit_direction='both', axis=0)
avg = (row + col) / 2

avg.to_csv('chr11_118375433-int.csv', index=True, sep=',', header=True)

# add excluding the 0 borders 
for row in range(1, len(df)-1):
    for col in range(1, len(df[0]-1)):
        rowIdx = (row-1) + ((col-1)*(nCols-2))
        f[rowIdx][mutCol['chr11_118375433']] = avg[row][col]

(42, 44)
